In [1]:
# import libraries
import os
import re
import requests
import tiktoken  # for counting tokens
import pandas as pd  # for storing text and embeddings data
import string  # for removing punctuation
import unicodedata  # for normalizing text
import urllib.request  # for downloading html
from bs4 import BeautifulSoup  # for parsing html
from collections import deque
from html.parser import HTMLParser
from openai import OpenAI  # for calling the OpenAI API
from scipy import spatial  # for calculating vector similarities for search
from tenacity import (
    retry,
    wait_random_exponential,
    stop_after_attempt,
)  # for retrying API calls
from urllib.parse import urlparse
from utils.embeddings_utils import cosine_similarity

# models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-4o"

In [2]:
# constants
domain = "app.bentobatch.com"
full_url = "https://app.bentobatch.com/"
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Crawl Web Pages

Given a list of URLs, crawl each page and return the content of the page.


In [3]:
# Regex pattern to match a URL
HTTP_URL_PATTERN = r"^http[s]*://.+"


# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])


# Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    user_agent = "Mozilla/5.0"
    headers = {"User-Agent": user_agent}

    # Try to open the URL and read the HTML
    try:
        request = urllib.request.Request(url=url, headers=headers)
        # Open the URL and read the HTML
        with urllib.request.urlopen(request) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get("Content-Type").startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode("utf-8")
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks


# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))


def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
        os.mkdir("text/")

    if not os.path.exists("text/" + local_domain + "/"):
        os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
        os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url)  # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open(
            "text/" + local_domain + "/" + url[8:].replace("/", "_") + ".txt", "w"
        ) as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # Get the content from the URL
            text = soup.get_text()

            # Write the data to the file
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

In [4]:
crawl(full_url)

https://app.bentobatch.com/
https://app.bentobatch.com/case/etf_ethereum_top10
https://app.bentobatch.com/case/aave_usdt_eth
https://app.bentobatch.com/case/scroll_airdrop_hunting_advanced
https://app.bentobatch.com/case/pendle_points
https://app.bentobatch.com/case/yearn_v3_usdt_a
https://app.bentobatch.com/case/zircuit_ethena_usdt
https://app.bentobatch.com/case/aave_eth_eth
https://app.bentobatch.com/case/ether_fi_liquid
https://app.bentobatch.com/case/yearn_v3_usdce
https://app.bentobatch.com/case/zora_famous_creators
https://app.bentobatch.com/case/zircuit_rsweth
https://app.bentobatch.com/case/kelp_and_pendle
https://app.bentobatch.com/case/yearn_v3_weth
https://app.bentobatch.com/case/aerodrome_degen_liquidity
https://app.bentobatch.com/case/scroll_airdrop_hunting_professional
https://app.bentobatch.com/case/ethena_usdt
https://app.bentobatch.com/case/ether_fi_arbitrum_bridge
https://app.bentobatch.com
https://app.bentobatch.com/case/zircuit_renzo
https://app.bentobatch.com/case

# Data Cleaning


In [20]:
def remove_newlines(serie):
    """Remove newlines from a pandas series for better processing."""
    serie = serie.str.replace("\n", " ")
    serie = serie.str.replace("\\n", " ")
    serie = serie.str.replace("  ", " ")
    serie = serie.str.replace("  ", " ")
    return serie

In [21]:
# Create a list to store the data records
data = []

# Get all the text files in the text directory
for file in os.listdir("text/" + domain + "/"):

    # Open the file and read the text
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()

        # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
        data.append(
            (
                file[len(domain) + 6 : -4]
                .replace("-", " ")
                .replace("_", " ")
                .replace("#update", ""),
                text,
            )
        )
# Create a dataframe from the list of texts
df = pd.DataFrame(data, columns=["fname", "text"])
df["text"] = df.fname + ". " + remove_newlines(df.text)
df.to_csv("processed/data.csv")

In [22]:
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model=EMBEDDING_MODEL):
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model)

    return response.data[0].embedding

In [23]:
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))
df.to_csv("processed/embeddings.csv")
df.head()

,fname,text,embedding
0,pendle points,pendle points. Bento Batch | One Click. All se...,"[-0.017359748482704163, -9.767817391548306e-05..."
1,zora most minted,zora most minted. Bento Batch | One Click. All...,"[-0.0002776383771561086, 0.0015714016044512391..."
2,genesis liquidity,genesis liquidity. Bento Batch | One Click. Al...,"[-0.006265711970627308, -0.005031391978263855,..."
3,scroll airdrop hunting rookie,scroll airdrop hunting rookie. Bento Batch | O...,"[-0.013905385509133339, -0.02448737621307373, ..."
4,zora famous creators,zora famous creators. Bento Batch | One Click....,"[0.018052520230412483, 0.0011609457433223724, ..."


In [24]:
def search_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: cosine_similarity(x, y),
    top_n: int = 100,
):
    # Get the embedding for the query
    query_embedding = get_embedding(query)

    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [25]:
# examples
# strings, relatednesses = search_functions("AAVE related", df, top_n=5)
# for string, relatedness in zip(strings, relatednesses):
#     print(f"{relatedness=:.3f}")
#     display(string)

# Ask


In [36]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatedness = search_relatedness(query, df)
    introduction = 'You are a member of BentoBatch. Use the provided data to answer the subsequent question. If the answer cannot be found in the data, write "I don\'t know."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        data = f'\n\nData:\n"""\n{string}\n"""'
        if num_tokens(message + data + question, model=model) > token_budget:
            break
        else:
            message += data
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {
            "role": "system",
            "content": "You answer questions about the cases on BentoBatch.",
        },
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message

### Examples


In [46]:
ask("What is BentoBatch?")

'Bento Batch is a Streamlined Transaction Layer (STL) that leverages Account Abstraction techniques to upscale blockchain efficiency, providing an easier, cheaper, and safer on-chain experience.'

In [47]:
ask("What is BentoBox?")

"BentoBox is a reward system in Bento Batch. Users can earn Bento Boxes by executing a Batch, with higher gas fees, more transactions per Batch, and larger volume input resulting in more Bento Boxes as rewards. Additionally, users can refer friends and receive an additional 10% of their friends' Bento Boxes as a bonus."

In [48]:
ask("Why do I need BentoBatch if I can interact with the dApps directly?")

'BentoBatch provides a streamlined transaction layer (STL) that consolidates multiple on-chain interactions into a single transaction. This means you no longer need to sign each transaction individually or understand every transaction detail. Instead, you define your objectives, and BentoBatch facilitates your needs efficiently with just one click. This makes the on-chain experience easier, cheaper, and safer.'

In [49]:
ask("What cases are related to AAVE?")

'The cases related to AAVE are:\n\n1. **aave eth eth. Bento Batch | One Click. All set.**\n   - Deposit ETH and borrow ETH on AAVE [Normal-Mode]\n   \n2. **aave eth eth emode. Bento Batch | One Click. All set.**\n   - Deposit ETH and borrow ETH on AAVE [E-Mode]\n   \n3. **aave usdc eth. Bento Batch | One Click. All set.**\n   - Deposit USDC to borrow ETH on AAVE\n   \n4. **aave usdt eth. Bento Batch | One Click. All set.**\n   - Deposit USDT to borrow ETH on AAVE'

In [50]:
ask("What cases are official collaborations?")

'The following cases are official collaborations:\n\n1. **Ether.fi**:\n   - Earn extra Ether.Fi points and BridgeEthereum\n   - Stake ETH to EtherFi and Zircuit to earn Zircuit points, EtherFi loyalty points, and EigenLayer Points\n\n2. **Swell**:\n   - Earn extra 15% Zircuit Points with Swell Restaking\n\n3. **Ethena**:\n   - Earn extra 15% Zircuit Points with Ethena Restaking by USDC\n   - Earn extra 15% Zircuit Points with Ethena Restaking by USDT\n\n4. **Renzo**:\n   - Earn extra 15% Zircuit Points with Renzo Restaking by ETH\n\n5. **Penpad**:\n   - Earn extra 10% Penpad Points and share up to 10000 $BLT prize pool\n\n6. **Genesis**:\n   - Earn extra 20% GenesisLRT restaking points and get x15 Gems boost\n\n7. **Zircuit**:\n   - Restake with Eigenpie & Swell & Zircuit'

In [51]:
ask("I have a plenty of usdc, what can I do with it?")

'You have several options for using your USDC on BentoBatch:\n\n1. **Deposit USDC to borrow ETH on AAVE**:\n   - Deposit USDC and borrow up to 70% equivalent value in ETH.\n   - Interact with AAVE contracts to supply USDC as collateral and borrow ETH.\n\n2. **Earn extra 15% Zircuit Points with Ethena Restaking by USDC**:\n   - Swap USDC to USDe and deposit USDe on Zircuit.\n   - Interact with Curve and Zircuit contracts for swapping and staking.\n\n3. **Yield farming on Yearn with USDC.e**:\n   - Deposit USDC.e to Yearn finance’s V3 Vault for auto compounding and high APY.\n   - Interact with Yearn V3 contracts to approve and deposit USDC.e.\n\n4. **Yield Farming on Yearn with Gamma USDC.e/WETH LP**:\n   - Provide USDC.e/WETH LP on Gamma and reinvest it on Yearn V3 vaults.\n   - Interact with 1inch, Gamma Vault, and Yearn V3 contracts for swapping, approving, and depositing.\n\n5. **On-chain Interaction on Scroll: Professional**:\n   - Swap, wrap, unwrap, lend, and borrow using USDC.\n

In [55]:
ask("What is the point system used for?")

'The point system on BentoBatch is used to reward users for engaging in various DeFi activities such as staking, restaking, swapping, lending, and providing liquidity. Users can earn different types of points like Kelp Miles, Renzo ezPoints, EigenLayer Points, Juice Finance Points, Zircuit Points, EtherFi loyalty points, Blast Points, and more, depending on the specific strategy or platform they interact with. These points can be accumulated and potentially used for benefits or rewards within the BentoBatch ecosystem.'

In [56]:
ask("Is it safe to use BentoBatch?")

"I don't know."

In [57]:
ask("How do you think this project should develop its tokenomics?")

"I don't know. The provided data does not contain any information or details about the tokenomics of the project."